# Predict when statistics need to be collected

## Connect to Vantage

In [1]:
#import the teradataml package for Vantage access
from teradataml import *
import getpass
from teradataml import display
#display.print_sqlmr_query=True
from sqlalchemy.sql.expression import select, case as case_when, func
from sqlalchemy import TypeDecorator, Integer, String
import warnings
warnings.filterwarnings('ignore')

In [2]:
Vantage = 'tdap1627t2.labs.teradata.com'
User = 'alice'
Pass = 'alice'

In [3]:
print(Vantage,User)

tdap1627t2.labs.teradata.com alice


In [40]:
con = create_context(Vantage, User, Pass)

## Get the Sentiment from the explains

In [16]:
dbqlog = DataFrame.from_table(in_schema("dbc", "dbqlogtbl")).drop("ZoneId", axis = 1)
dbqlexplain = DataFrame.from_table(in_schema("dbc", "dbqlexplaintbl")).drop("ZoneID", axis = 1)
dbqldata = dbqlog.join(other = dbqlexplain, on = ["QueryID"], lsuffix = "t1", rsuffix = "t2") \
    .select(['t1_QueryID','ExplainText','QueryBand','QueryText'])

In [17]:
dbqldata

               t1_QueryID                                        ExplainText QueryBand                                          QueryText
0  307190295502502401.000    1) First, we do an INSERT step into R_GLM_WO...      None  insert into R_GLM_WORKFLOWS_DB.r_music values ...
1  307180295502824652.000    1) First, we lock ALICE.cars_hist in TD_MAP1...      None  CREATE MULTISET TABLE "ALICE".r__t__sqlmr_stdo...
2  307170295502603825.000    1) First, we do an INSERT step into ALICE.ci...      None            insert into citvertices_2 values(2064);
3  307160295502627742.000    1) First, we do an INSERT step into ALICE.co...      None  insert into computers_train1 values(1842,2995,...
4  307170295502609392.000    1) First, we do an INSERT step into ALICE.su...      None  insert into surveys values(4583,6,5,1981,11,'O...
5  307180295502804933.000    1) First, we do an INSERT step into ALICE.su...      None  insert into surveys values(18984,8,8,1991,6,'P...
6  307170295502629111.000    1) Fi

In [18]:
# Workaround until ELE-2072.
dbqldata.to_sql('prediction_sentiment', if_exists="replace")
dbqldata = DataFrame.from_table('prediction_sentiment')

In [19]:
df_select_query_column_projection = [
     dbqldata.t1_QueryID.expression.label("queryid"),
     dbqldata.ExplainText.expression.label("explaintext"),
     dbqldata.QueryBand.expression.label("queryband"),
     func.REGEXP_SUBSTR(dbqldata.QueryBand.expression, 
                        '(collected_statistics|no_statistics)', 1, 1, 'i').label("training"),
     func.REGEXP_SUBSTR(dbqldata.QueryText.expression, 
                        'SELECT', 1, 1, 'i').label("select_info"),
     func.REGEXP_SUBSTR(func.REGEXP_SUBSTR(dbqldata.ExplainText.expression, 
                        '(joined using a *[A-z \-]+ join,)', 1, 1, 'i'), 
                            '[A-z]+', 15, 1, 'i').label("join_condition")]

In [20]:
prediction_data = DataFrame.from_query(str(select(df_select_query_column_projection)
                                 #.where(Column('join_condition') != None)
                                 #.where(Column('training') != None)
                                 .compile(compile_kwargs={"literal_binds": True})))

In [25]:
data_set = (prediction_data.join_condition != None)  & (prediction_data.training != None)
prediction_set = prediction_data[data_set]

In [26]:
prediction_data.select(['queryid', 'join_condition', 'explaintext', 'training'])
# Workaround until ELE-2072.
#prediction_set.to_sql('prediction_sentiment')
#prediction_set = DataFrame.from_table('prediction_sentiment')
prediction_set

                  queryid                                        explaintext                                  queryband              training select_info join_condition
0  307170295502596206.000    1) First, we lock DBC.udt1 in view ColumnsV ...         =S> run=#jdf_8_00_2_no_statistics;         no_statistics      SELECT          merge
1  307160295502622239.000    1) First, we lock DBC.DBase in view All_RI_C...         =S> run=#jdf_8_00_3_no_statistics;         no_statistics      SELECT        product
2  307170295502596173.000    1) First, we lock DBC.DBase in view All_RI_C...         =S> run=#jdf_8_00_1_no_statistics;         no_statistics      SELECT        product
3  307160295502622236.000    1) First, we lock DBC.udt1 in view ColumnsV ...         =S> run=#jdf_8_00_3_no_statistics;         no_statistics      SELECT          merge
4  307160295502622374.000    1) First, we lock alice.d2 in TD_MAP1 for re...  =S> run=#jdf_8_84_7_collected_statistics;  collected_statistics      select  

In [27]:
dictionary = DataFrame.from_table('dbql_sentiment')

In [28]:
td_sentiment_extractor_out = SentimentExtractor(
    dict_data = dictionary,
    newdata = prediction_set,
    level = "document",
    text_column = "explaintext",
    accumulate = ['queryid','join_condition','training']
)

In [29]:
predict = td_sentiment_extractor_out.result #.to_sql('holdit4')

In [30]:
predict

              queryid join_condition              training out_polarity  out_strength                                out_sentiment_words
0  307160295502622374        dynamic  collected_statistics          POS             2  high confidence 10, low confidence -5, high co...
1  307170295502596205          merge         no_statistics          NEG             2  low confidence -5, no confidence -10, no confi...
2  307170295502596172         nested         no_statistics          NEG             2  low confidence -5, no confidence -10, no confi...
3  307170295502596170          merge         no_statistics          NEG             2  low confidence -5, no confidence -10, no confi...
4  307170295502596209        product         no_statistics          NEG             2  no confidence -10, no confidence -10, no confi...
5  307160295502622301        dynamic  collected_statistics          POS             2  high confidence 10, low confidence -5, high co...
6  307160295502622235          merge     

In [31]:
try:
    con.execute("drop table target_collection")
except:
    pass

In [32]:
stats_model = DataFrame.from_table(in_schema("alice", "stats_model"))

# Why does it need formula?

In [36]:
# Predict from queries columns needing collected statistics
target_collection = NaiveBayesPredict(newdata=predict,
                                       modeldata = stats_model,
                                       formula="training ~ out_polarity + join_condition", 
                                       id_col = "queryid",
                                       responses = ["collected_statistics","no_statistics"]
                                       ).result

In [37]:
target_collection.result.to_sql('acc1', if_exists="replace")

In [47]:
target_collection.result

              queryid     prediction  loglik_no_statistics
0  307160295502622374  no_statistics             -3.283414
1  307170295502596205  no_statistics             -0.916291
2  307170295502596172  no_statistics             -1.203973
3  307170295502596170  no_statistics             -0.916291
4  307170295502596209  no_statistics             -2.302585
5  307160295502622301  no_statistics             -3.283414
6  307160295502622235  no_statistics             -0.916291
7  307170295502596169  no_statistics             -0.916291
8  307160295502622264  no_statistics             -3.283414
9  307160295502622236  no_statistics             -0.916291

In [55]:
dbqlobj = DataFrame.from_table('dbc.dbqlobjtbl')

                  queryid
0  307190295502491943.000
1  307190295502491944.000
2  307190295502491944.000
3  307190295502491944.000
4  307190295502491952.000
5  307190295502491952.000
6  307190295502491944.000
7  307190295502491943.000
8  307190295502491943.000
9  307190295502491943.000

In [ ]:
# Obtain query's join information
target_names = target_collection.result.join(other = dbqlobj, on = ["queryid"], lsuffix = "t1", 
        rsuffix = "t2").select('objectdatabasename', 'objecttablename', 'objectcolumnname')

In [ ]:
# Collect statistics on each column
for index, row in target_collection.result.to_pandas().iterrows():
    con.execute('collect statistics column '+row['ObjectTableName']+" on "+ \
        row['ObjectDatabaseName']+'.'+row['ObjectTableName'])

In [ ]:
## how to test if table is still there, no help table.

In [41]:
statement

['collect statistics column DATA2 on alice.DATA2',
 'collect statistics column DATA1 on alice.DATA1']